# AI E-commerce Shopping Assistant

AI E-commerce Shopping Assistant using PraisonAI - A conversational commerce agent that helps customers discover products, create orders, and manage

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DhivyaBharathy-web/PraisonAI/blob/main/examples/cookbooks/ai_ecommerce_shopping_assistant.ipynb)


In [1]:
# @title **📦 Installation & Dependencies**

# Install required packages
!pip install praisonaiagents openai python-dotenv duckduckgo-search -q

# Basic imports
import os
import json
import time
from typing import Dict, List, Any
from datetime import datetime
import getpass

print("📦 Dependencies installed!")
print("🔄 Using PraisonAI Framework")
print("🛒 E-commerce shopping agent ready!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.2/178.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
📦 Dependencies installed!
🔄 Using PraisonAI Framework
🛒 E-commerce shopping agent ready!


In [6]:
# @title **🔑 API Key Configuration**

# Set OpenAI API key (REQUIRED before importing praisonaiagents)
openai_api_key = getpass.getpass("🔑 Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

# Verify API key is set
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("❌ OpenAI API Key is required!")

print("✅ OpenAI API Key configured!")

# Now import PraisonAI (after API key is set)
from praisonaiagents import Agent, Task, PraisonAIAgents

print("✅ PraisonAI imported successfully!")
print("🛒 Ready to create shopping assistant!")

🔑 Enter your OpenAI API Key: ··········
✅ OpenAI API Key configured!
✅ PraisonAI imported successfully!
🛒 Ready to create shopping assistant!


In [5]:
# @title **🛍️ E-commerce Tools (Demo Database)**

class EcommerceDatabase:
    """Demo e-commerce database for shopping assistant"""

    def __init__(self):
        self.products = [
            {
                "id": "1",
                "name": "Premium Wireless Headphones",
                "price": 199.99,
                "category": "electronics",
                "description": "High-quality wireless headphones with noise cancellation",
                "stock": 25,
                "brand": "TechPro"
            },
            {
                "id": "2",
                "name": "Organic Coffee Blend",
                "price": 24.99,
                "category": "food",
                "description": "Fair trade organic coffee beans, medium roast",
                "stock": 50,
                "weight": "500g"
            },
            {
                "id": "3",
                "name": "Fitness Tracker Watch",
                "price": 129.99,
                "category": "electronics",
                "description": "Smart fitness tracker with heart rate monitor",
                "stock": 15,
                "brand": "FitLife"
            },
            {
                "id": "4",
                "name": "Eco-Friendly Water Bottle",
                "price": 29.99,
                "category": "lifestyle",
                "description": "Stainless steel water bottle, keeps drinks cold for 24hrs",
                "stock": 40,
                "capacity": "750ml"
            },
            {
                "id": "5",
                "name": "Premium Tea Collection",
                "price": 34.99,
                "category": "food",
                "description": "Assorted premium teas from around the world",
                "stock": 30,
                "items": "12 tea bags"
            }
        ]

        self.orders = []
        self.order_counter = 1

    def get_all_products(self):
        """Get all available products"""
        return json.dumps(self.products, indent=2)

    def get_product_by_id(self, product_id: str):
        """Get specific product by ID"""
        for product in self.products:
            if product["id"] == product_id:
                return json.dumps(product, indent=2)
        return json.dumps({"error": "Product not found"})

    def search_products(self, category: str = None, name: str = None):
        """Search products by category or name"""
        results = self.products

        if category:
            results = [p for p in results if category.lower() in p["category"].lower()]

        if name:
            results = [p for p in results if name.lower() in p["name"].lower()]

        return json.dumps(results, indent=2)

    def create_order(self, products: List[Dict], customer_name: str, customer_email: str):
        """Create a new order"""
        order_items = []
        total_amount = 0

        for item in products:
            product_id = item.get("productId")
            quantity = item.get("quantity", 1)

            # Find product
            product = None
            for p in self.products:
                if p["id"] == product_id:
                    product = p
                    break

            if not product:
                return json.dumps({"error": f"Product {product_id} not found"})

            if product["stock"] < quantity:
                return json.dumps({"error": f"Insufficient stock for {product['name']}"})

            item_total = product["price"] * quantity
            total_amount += item_total

            order_items.append({
                "productId": product_id,
                "productName": product["name"],
                "quantity": quantity,
                "price": product["price"],
                "total": item_total
            })

            # Update stock
            product["stock"] -= quantity

        order = {
            "orderId": str(self.order_counter),
            "customer": {
                "name": customer_name,
                "email": customer_email
            },
            "items": order_items,
            "totalAmount": total_amount,
            "status": "pending",
            "paymentLink": f"https://payments.store.com/pay/{self.order_counter}",
            "createdAt": datetime.now().isoformat()
        }

        self.orders.append(order)
        self.order_counter += 1

        return json.dumps(order, indent=2)

    def get_orders(self, customer_email: str = None):
        """Get orders, optionally filtered by customer"""
        if customer_email:
            filtered_orders = [o for o in self.orders if o["customer"]["email"] == customer_email]
            return json.dumps(filtered_orders, indent=2)
        return json.dumps(self.orders, indent=2)

# Initialize e-commerce database
ecommerce_db = EcommerceDatabase()

# Define tool functions for PraisonAI
def get_products() -> str:
    """Get all available products from the online store"""
    return ecommerce_db.get_all_products()

def get_product(product_id: str) -> str:
    """Get specific product details by ID"""
    return ecommerce_db.get_product_by_id(product_id)

def search_products(category: str = None, name: str = None) -> str:
    """Search products by category or name"""
    return ecommerce_db.search_products(category, name)

def create_order(products: List[Dict[str, Any]], customer_name: str, customer_email: str) -> str:
    """
    Create a new order with payment link.

    Args:
        products: List of {"productId": "1", "quantity": 2}
        customer_name: Customer's full name
        customer_email: Customer's email address
    """
    return ecommerce_db.create_order(products, customer_name, customer_email)

def get_customer_orders(customer_email: str) -> str:
    """Get all orders for a specific customer"""
    return ecommerce_db.get_orders(customer_email)

def book_meeting() -> str:
    """Provide meeting booking information for customer support"""
    return json.dumps({
        "message": "To speak with a human representative, you can book a meeting here:",
        "meeting_link": "https://calendly.com/customer-support",
        "availability": "Monday-Friday, 9 AM - 6 PM EST",
        "response_time": "We'll respond within 24 hours"
    })

print("🛍️ E-commerce tools initialized!")
print("   📦 Product management")
print("   🛒 Order processing")
print("   💳 Payment link generation")
print("   🤝 Customer support booking")

🛍️ E-commerce tools initialized!
   📦 Product management
   🛒 Order processing
   💳 Payment link generation
   🤝 Customer support booking


In [7]:
# @title **🤖 AI Shopping Assistant Agent**

def create_shopping_agent():
    """Create the AI shopping assistant agent"""

    shopping_assistant = Agent(
        name="Shopping Assistant",
        role="AI E-commerce Assistant",
        goal="Help customers discover products, manage orders, and provide excellent shopping experience through natural conversation",
        backstory=f"""You are a friendly AI shopping assistant for an online store that offers
        a variety of products including electronics, food items, and lifestyle products.

        You help customers find products, create orders, and provide customer support through
        natural conversation. You are knowledgeable about all products in the store and can
        make recommendations based on customer needs.

        Available product categories:
        - Electronics (headphones, fitness trackers, tech gadgets)
        - Food & Beverages (coffee, tea, organic products)
        - Lifestyle (water bottles, eco-friendly products)

        You are helpful, conversational, and always guide customers through their shopping
        journey with clear next steps. Today is {datetime.now().strftime('%B %d, %Y')}.

        Key responsibilities:
        1. Help customers discover products that match their needs
        2. Provide detailed product information and recommendations
        3. Create orders and generate secure payment links
        4. Track existing orders and provide status updates
        5. Connect customers with human support when needed

        Always be friendly, helpful, and clear in your communication.""",

        tools=[
            get_products,
            get_product,
            search_products,
            create_order,
            get_customer_orders,
            book_meeting
        ],

        verbose=True,
        allow_delegation=False
    )

    return shopping_assistant

# Create shopping assistant agent
shopping_agent = create_shopping_agent()

print("🤖 Shopping Assistant Agent Created!")
print("   🛒 Ready to assist with shopping")
print("   📦 Product discovery and recommendations")
print("   💳 Order creation and payment processing")
print("   🤝 Customer support integration")

🤖 Shopping Assistant Agent Created!
   🛒 Ready to assist with shopping
   📦 Product discovery and recommendations
   💳 Order creation and payment processing
   🤝 Customer support integration


In [8]:
# @title **📋 Shopping Assistant Tasks**

def create_shopping_tasks():
    """Create tasks for the shopping assistant"""

    tasks = []

    # Product Discovery Task
    tasks.append(Task(
        description="""Help a customer discover products in the online store.
        Show available products, provide recommendations based on preferences,
        and explain product details including prices and descriptions.

        Example interactions:
        - "What electronics do you have?"
        - "Show me coffee products"
        - "I need a gift for someone who likes tech"
        """,
        agent=shopping_agent,
        expected_output="Product recommendations with details, prices, and helpful information"
    ))

    # Order Management Task
    tasks.append(Task(
        description="""Assist customers with creating orders and managing their purchases.
        Guide them through the ordering process, collect necessary information,
        and generate payment links for secure checkout.

        Example interactions:
        - "I want to order 2 wireless headphones"
        - "Create an order for Jane Doe, jane@email.com"
        - "Check my order status"
        """,
        agent=shopping_agent,
        expected_output="Successfully created orders with payment links or order status information"
    ))

    # Customer Support Task
    tasks.append(Task(
        description="""Provide excellent customer support including answering questions
        about the store, helping with product inquiries, and connecting customers
        with human representatives when needed.

        Example interactions:
        - "Tell me about your store"
        - "I need to speak with someone"
        - "What's your return policy?"
        """,
        agent=shopping_agent,
        expected_output="Helpful customer support responses and meeting booking information when requested"
    ))

    return tasks

# Create shopping tasks
shopping_tasks = create_shopping_tasks()

print("📋 Shopping Tasks Created!")
print(f"   📊 Total Tasks: {len(shopping_tasks)}")
print("   🔍 Product Discovery")
print("   🛒 Order Management")
print("   🤝 Customer Support")

📋 Shopping Tasks Created!
   📊 Total Tasks: 3
   🔍 Product Discovery
   🛒 Order Management
   🤝 Customer Support


In [9]:
# @title **🚀 Launch AI Shopping Assistant**

def run_shopping_assistant():
    """Launch the AI shopping assistant"""

    print("🛒 Starting AI Shopping Assistant...")
    print("=" * 50)

    # Initialize PraisonAI system
    shopping_system = PraisonAIAgents(
        agents=[shopping_agent],
        tasks=shopping_tasks,
        verbose=True,
        process="sequential"
    )

    # Execute the shopping assistant
    start_time = time.time()

    print("🤖 Activating Shopping Assistant...")
    result = shopping_system.start()

    end_time = time.time()
    duration = end_time - start_time

    print("=" * 50)
    print(f"✅ AI Shopping Assistant Ready!")
    print(f"⏱️ Startup Time: {duration:.2f} seconds")
    print(f"🛒 Online Store: ACTIVE")
    print(f"🤖 Agent Status: READY")

    return result

# Launch shopping assistant
shopping_result = run_shopping_assistant()

print("\n🎊 AI SHOPPING ASSISTANT ACTIVE!")
print("🛒 Ready to help customers with natural language shopping")
print("🤖 Powered by PraisonAI + OpenAI gpt-5-nano")

🛒 Starting AI Shopping Assistant...
🤖 Activating Shopping Assistant...


╭─ Agent Info ────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  👤 Agent: Shopping Assistant                                                                                   │
│  Role: AI E-commerce Assistant                                                                                  │
│  Tools: get_products, get_product, search_products, create_order, get_customer_orders, book_meeting             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────── Tool Call ───────────────────────────────────╮
│ Agent Shopping Assistant is calling function 'get_products' with arguments: {} │
╰────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Tool Call ───────────────────────────────────────────────────╮
│ Function 'get_products' returned: "[\n  {\n    \"id\": \"1\",\n    \"name\": \"Premium Wireless Headphones\",\n │
│ \"price\": 199.99,\n    \"category\": \"electronics\",\n    \"description\": \"High-quality wireless headphones │
│ with noise cancellation\",\n    \"stock\": 25,\n    \"brand\": \"TechPro\"\n  },\n  {\n    \"id\": \"2\",\n     │
│ \"name\": \"Organic Coffee Blend\",\n    \"price\": 24.99,\n    \"category\": \"food\",\n    \"description\":   │
│ \"Fair trade organic coffee beans, medium roast\",\n    \"stock\": 50,\n    \"weight\": \"500g\"\n  },\n  {\n   │
│ \"id\": \"3\",\n    \"name\": \"Fitness Tracker Watch\",\n    \"price\": 129.99,\n    \"category\":             │
│ \"electronics\",\n    \"description\": \"Smart fitness tracker with heart rate monitor\",\n    \"stock\": 15,\n │
│ \"brand\": \"FitLife\"\n  },\n  {\n    \"id\": \"4\",\n    \"name\": \"Eco-Friendly Water Bottle\",\n           │
│ \"price\": 29.99,\n    \"category\": \"lifestyle\",\n    \"description\": \"Stainless steel water bottle, keeps │
│ drinks cold for 24hrs\",\n    \"stock\": 40,\n    \"capacity\": \"750ml\"\n  },\n  {\n    \"id\": \"5\",\n      │
│ \"name\": \"Premium Tea Collection\",\n    \"price\": 34.99,\n    \"category\": \"food\",\n    \"description\": │
│ \"Assorted premium teas from around the world\",\n    \"stock\": 30,\n    \"items\": \"12 tea bags\"\n  }\n]"   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

Response generated in 4.4s

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│ You need to do the following task: Provide excellent customer support including answering questions about the   │
│ store, helping with product inquiries, and connecting customers with human representatives when needed.         │
│                                                                                                                 │
│                                                                                                                 │
│      Example interactions:                                                                                      │
│      - "Tell me about your store"                                                                               │
│      - "I need to speak with someone"                                                                           │
│      - "What's your return policy?"                                                                             │
│      .                                                                                                          │
│                                                                                                                 │
│                                                                                                                 │
│ Expected Output: Helpful customer support responses and meeting booking information when requested.             │
│                                                                                                                 │
│ Context:                                                                                                        │
│                                                                                                                 │
│ Result of previous task Assist customers with creating orders and managing their purchases. Guide them through  │
│ the ordering process, collect necessary information, and generate payment links for secure checkout.            │
│                                                                                                                 │
│                                                                                                                 │
│      Example interactions:                                                                                      │
│      - "I want to order 2 wireless headphones"                                                                  │
│      - "Create an order for Jane Doe, jane@email.com"                                                           │
│      - "Check my order status"                                                                                  │
│      :                                                                                                          │
│                                                                                                                 │
│                                                                                                                 │
│ To proceed with creating an order, I need the following information:                                            │
│                                                                                                                 │
│  1 Product(s) you wish to order along with the quantity.                                                        │
│  2 Customer's full name.                                                                                        │
│  3 Customer's email address.                                                                                    │
│                                                                                                                 │
│ Once I have this information, I can generate a secure payment link for you. Please provide only the final       │
│ result of your work. Do not add any conversation or ex

╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│ Customer Support Responses:                                                                                     │
│                                                                                                                 │
│  1 About Our Store:                                                                                             │
│     • We are an online store offering a wide range of products in electronics, food & beverages, and lifestyle  │
│       categories. Our mission is to provide high-quality products with excellent customer service.              │
│  2 Speaking with a Representative:                                                                              │
│     • If you need to speak with a human representative, I can help you book a meeting for personalized          │
│       assistance.                                                                                               │
│  3 Return Policy:                                                                                               │
│     • We offer a 30-day return policy for most products. Items must be returned in their original condition     │
│       with proof of purchase. Please contact our support team for further assistance with returns.              │
│                                                                                                                 │
│ Meeting Booking Information:                                                                                    │
│                                                                                                                 │
│  • To book a meeting with a customer support representative, please let me know your preferred date and time,   │
│    and I will arrange it for you.                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ AI Shopping Assistant Ready!
⏱️ Startup Time: 15.13 seconds
🛒 Online Store: ACTIVE
🤖 Agent Status: READY

🎊 AI SHOPPING ASSISTANT ACTIVE!
🛒 Ready to help customers with natural language shopping
🤖 Powered by PraisonAI + OpenAI gpt-5-nano


In [10]:
# @title **🎮 Interactive Shopping Demo**

def demo_shopping_conversation():
    """Demonstrate AI shopping assistant capabilities"""

    print("🎮 AI SHOPPING ASSISTANT DEMO")
    print("=" * 40)

    # Simulate customer interactions
    demo_conversations = [
        {
            "customer": "Hi! What electronics do you have?",
            "action": "product_discovery"
        },
        {
            "customer": "I want to order 1 fitness tracker for Alice Johnson, alice@email.com",
            "action": "order_creation"
        },
        {
            "customer": "Can I speak with a human representative?",
            "action": "customer_support"
        }
    ]

    for i, conversation in enumerate(demo_conversations, 1):
        print(f"\n{i}. 👤 Customer: {conversation['customer']}")
        print(f"   🤖 AI Action: {conversation['action']}")

        if conversation['action'] == 'product_discovery':
            products = search_products(category="electronics")
            print("   📦 Showing electronics products...")

        elif conversation['action'] == 'order_creation':
            sample_order = create_order(
                [{"productId": "3", "quantity": 1}],
                "Alice Johnson",
                "alice@email.com"
            )
            print("   🛒 Order created with payment link!")

        elif conversation['action'] == 'customer_support':
            support_info = book_meeting()
            print("   🤝 Meeting booking information provided!")

    print(f"\n✅ Demo completed successfully!")
    print("🛒 AI can handle: Product discovery, Order creation, Customer support")

    # Show current store status
    print(f"\n📊 STORE STATUS:")
    print(f"   📦 Products Available: {len(ecommerce_db.products)}")
    print(f"   🛒 Orders Created: {len(ecommerce_db.orders)}")
    print(f"   💰 Store Status: ONLINE")

# Run demo
demo_shopping_conversation()

print("\n" + "=" * 50)
print("🎊 AI SHOPPING ASSISTANT DEMO COMPLETE!")
print("🛒 Conversational E-commerce Agent")
print("🤖 Powered by PraisonAI + OpenAI")
print("✨ Ready for Real-World E-commerce!")
print("=" * 50)

🎮 AI SHOPPING ASSISTANT DEMO

1. 👤 Customer: Hi! What electronics do you have?
   🤖 AI Action: product_discovery
   📦 Showing electronics products...

2. 👤 Customer: I want to order 1 fitness tracker for Alice Johnson, alice@email.com
   🤖 AI Action: order_creation
   🛒 Order created with payment link!

3. 👤 Customer: Can I speak with a human representative?
   🤖 AI Action: customer_support
   🤝 Meeting booking information provided!

✅ Demo completed successfully!
🛒 AI can handle: Product discovery, Order creation, Customer support

📊 STORE STATUS:
   📦 Products Available: 5
   🛒 Orders Created: 1
   💰 Store Status: ONLINE

🎊 AI SHOPPING ASSISTANT DEMO COMPLETE!
🛒 Conversational E-commerce Agent
🤖 Powered by PraisonAI + OpenAI
✨ Ready for Real-World E-commerce!
